In [ ]:
import pip
pip.main(["install", "openpyxl"])

In [1]:
import pandas as pd
# import openpyxl

In [50]:
df = pd.read_excel("../../data/production/WMD2022 Commodities 1984-2020 extraInfo_Litio_USGS.xlsx")
df

,Minerals in metric tons*,1984,1985,1986,1987,1988,1989,1990,1991,1992,...,2015,2016,2017,2018,2019,2020,Es inicio,Kilo,cmdCode,Version
0,Iron Ore (Fe-cont.),446887838.0,492697409.0,495584899.0,507784666.0,524316505.0,539637227.0,529023870.0,531831265.0,499087427.0,...,1.456311e+09,1.466177e+09,1.511759e+09,1.523930e+09,1.552366e+09,1.522559e+09,1.0,NaN,2601.0,org
1,Albania,439502.0,414035.0,379154.0,444690.0,488050.0,539314.0,425697.0,203830.0,10000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,org
2,Algeria,2000000.0,1836000.0,1836000.0,1836000.0,1674000.0,1483920.0,1588140.0,1266000.0,1362400.0,...,5.098000e+05,3.283000e+05,2.684000e+05,3.812000e+05,4.099000e+05,4.066000e+05,NaN,NaN,NaN,org
3,Argentina,345576.0,388567.0,514259.0,359902.0,378576.0,414425.0,444115.0,88932.0,2502.0,...,2.060600e+05,1.072800e+05,2.840000e+04,4.056000e+04,0.000000e+00,0.000000e+00,NaN,NaN,NaN,org
4,Australia,56885000.0,62042117.0,60081983.0,64798343.0,61058000.0,66660300.0,66402630.0,74133990.0,72075000.0,...,5.009936e+08,5.310754e+08,5.482971e+08,5.600546e+08,5.679571e+08,5.683846e+08,NaN,NaN,NaN,org
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2674,* Minerals in metric tons except,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2675,"Rhenium, Precious Metals in kg",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Minerals in metric tons except
### Rhenium, Precious Metals in kg
### Natural Gas in Million m3


In [ ]:
# Get headers
header_indexes = df.index[df["Es inicio"] == 1.0].tolist()

def convert_df(ugly_df):
    # We get
    #   - name of the material
    #   - commodity code
    #   - if we have to handle it as kilogram or 1,000 kg
    #   - if it's the second version of lithium (USGS)
    # from first column and first row
    mineral_name, cmdCode, isKg, source = ugly_df.iloc[0, 0], ugly_df.iloc[0]["cmdCode"], ugly_df.iloc[0]["Kilo"], ugly_df.iloc[0]["Version"]

    # Get rid of "Es inicio" column
    del ugly_df["Es inicio"]
    del ugly_df["Kilo"]
    del ugly_df["cmdCode"]
    del ugly_df["Version"]

    # Remove the 1st cell since it's a header
    tmp_df = ugly_df.iloc[1:]
    # Add column with the name of mineral/thing
    tmp_df["Product"] = mineral_name
    tmp_df["cmdCode"] = cmdCode
    tmp_df["Source"] = source

    # Change the 1st column as Country
    tmp_df.rename(columns={"Minerals in metric tons*": "Country"}, inplace=True)
    # Melt the df into a conventional table    
    melted_df = tmp_df.melt(
        id_vars=["Country", "Product", "cmdCode", "Source"],
        var_name="Year",
        value_name="Value"
    )
    # If isKg flag not equals to 1 that means we have to multiply it by 1,000 because of is considered as a ton
    if isKg != 1:
        melted_df["Value (Ton)"] = melted_df["Value"]
        melted_df["Value"] = melted_df["Value"] * 1000
    else:
        melted_df["Value (Ton)"] = melted_df["Value"] / 1000
    return melted_df

list_df = list()
for n in range(len(header_indexes) - 1):
    # print(header_indexes[n], header_indexes[n + 1])
    tmp_df = df.iloc[header_indexes[n]:header_indexes[n + 1]]
    melted_df = convert_df(tmp_df)
    list_df.append(melted_df)
# and the last one. Skip last 5 rows because of the annotations
tmp_df = df.iloc[header_indexes[-1]:df.shape[0] - 5]
melted_df = convert_df(tmp_df)
list_df.append(melted_df)


In [ ]:
list_df[0]

In [ ]:
list_df[27]

In [73]:
list_df[-1]

,Country,Product,cmdCode,Source,Year,Value,Value (Ton)
0,Argentina,Uranium (U3O8-cont.),261210.0,org,1984,151000.0,151.0
1,Australia,Uranium (U3O8-cont.),261210.0,org,1984,5099000.0,5099.0
2,Belgium,Uranium (U3O8-cont.),261210.0,org,1984,59000.0,59.0
3,Brazil,Uranium (U3O8-cont.),261210.0,org,1984,138000.0,138.0
4,Bulgaria,Uranium (U3O8-cont.),261210.0,org,1984,0.0,0.0
...,...,...,...,...,...,...,...
1179,Spain,Uranium (U3O8-cont.),261210.0,org,2020,NaN,NaN
1180,Ukraine,Uranium (U3O8-cont.),261210.0,org,2020,877000.0,877.0
1181,United States,Uranium (U3O8-cont.),261210.0,org,2020,7000.0,7.0
1182,Uzbekistan,Uranium (U3O8-cont.),261210.0,org,2020,4127000.0,4127.0


In [71]:
len(list_df)

67

## Convert the list of dataframes into only one DF and save it as CSV

In [65]:
final_production_df = pd.concat(list_df, ignore_index=True)

In [67]:
final_production_df.to_csv("../../data/production/Production_one_file_2.csv", index=False)

In [72]:
len(final_production_df["Country"].unique()) * 37 * 67

443741